In [ ]:
!pip install -q optuna

In [ ]:
def optimize_lgb(trial):
    params = {
        'max_depth' : trial.suggest_int('max_depth', 2, 11),
        'scale_pos_weight' : trial.suggest_float('scale_pos_weight', def_scale_pos_weight/2, def_scale_pos_weight*2),
        'reg_lambda' : trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        'reg_alpha' : trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True)
    }
    if params['max_depth'] == 11:
        params['max_depth'] = -1

    # 모델 원형
    clf = lgb.LGBMClassifier(random_state=rand, n_jobs=-1, **params) # 이중 참조
    clf.fit(X_train, y_train)

    metrics_dict = evaluate_class_mdl(clf, X_train, X_test, y_train, y_test, plot=False)

    # optimize를 위한 최소조건을 준다는 의미
    if (metrics_dict['tn%'] < min_tn) or (metrics_dict['fp%'] > max_fp):
        return 0

    return metrics_dict['recall']

%%time
opt_study = optuna.create_study(direction='maximize')
opt_study.optimize(optimize_lgb, n_trials=100)

# best_params
opt_study.best_params